## Proyecto para extraer comentarios de un post IG usando Selenium

In [35]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, urllib.request
import pandas as pd
import requests
import datetime

### Cambiar por la dirección en donde se encuentre almacenado el driver(el del proyecto es de linux)
[Link de descarga](https://chromedriver.chromium.org/)

In [36]:
PATH = r"/home/kevinmaxi/PycharmProjects/instagramScrapping/drivers/chromedriver"
driver = webdriver.Chrome(PATH)

driver.get("https://www.instagram.com/")


### Cambiar por los datos de inicio de sesión de una cuenta de IG (de preferencia una secundaria, por posibles baneos)

In [ ]:
username = "nombreDeUsuario"
password = "contraseña"
postURL = "https://www.instagram.com/p/B166OkVBPJR/"

### Inicio de sesión

In [37]:
time.sleep(5)
username=driver.find_element_by_css_selector("input[name='username']")
password=driver.find_element_by_css_selector("input[name='password']")
username.clear()
password.clear()
username.send_keys(username)
password.send_keys(password)
login = driver.find_element_by_css_selector("button[type='submit']").click()
time.sleep(10)
notnow = driver.find_element_by_xpath("//button[contains(text(), 'Not Now')]").click()
#turn on notif
time.sleep(10)
notnow2 = driver.find_element_by_xpath("//button[contains(text(), 'Not Now')]").click()


### Proceso de extracción
Los timers son puestos para evitar el baneo del scrapper

In [38]:
time.sleep(5)
driver.get(postURL)


### Al parecer hay un error al sobrecargar la página y se recibe como que no existe el botón para seguir expandiendo los comentarios
### para solucionarlo se usa la diferencia de tiempo que hay entre el error de no encontar el elemento y el de la última ejeución, si el el tiempo lo excede quiere decir que no hay forma de obtener mas comentarios y el bucle para

In [39]:
time.sleep(3)
seguir = True
threshold = datetime.timedelta(seconds=7)
num_max_fallos=20
fallos=0
time_fail = datetime.datetime.now()+threshold
now = datetime.datetime.now()+5*threshold
# print("Hola")
while seguir:
    # print("Ha entrado")
    try:
        load_more_comment = driver.find_element_by_css_selector('.MGdpg > button:nth-child(1)')
        i = 0
        while load_more_comment.is_displayed():
            # print("Desde aqui")
            load_more_comment.click()
            time.sleep(1.5)
            now = datetime.datetime.now()
            load_more_comment = driver.find_element_by_css_selector('.MGdpg > button:nth-child(1)')

            # print("Found {}".format(str(load_more_comment)))
            i += 1

    except Exception as e:

        time_elapsed = now - time_fail
        print(time_elapsed)
        print(threshold > time_elapsed)
        if threshold > time_elapsed:
            seguir = False
        time_fail = datetime.datetime.now()
        time.sleep(5)
        print(e)
        pass


0:01:59.038028
False
Message: no such element: Unable to locate element: {"method":"css selector","selector":".MGdpg > button:nth-child(1)"}
  (Session info: chrome=93.0.4577.82)

0:00:24.893726
False
Message: no such element: Unable to locate element: {"method":"css selector","selector":".MGdpg > button:nth-child(1)"}
  (Session info: chrome=93.0.4577.82)

0:00:14.072781
False
Message: no such element: Unable to locate element: {"method":"css selector","selector":".MGdpg > button:nth-child(1)"}
  (Session info: chrome=93.0.4577.82)

0:00:09.678870
False
Message: no such element: Unable to locate element: {"method":"css selector","selector":".MGdpg > button:nth-child(1)"}
  (Session info: chrome=93.0.4577.82)

0:00:28.584918
False
Message: no such element: Unable to locate element: {"method":"css selector","selector":".MGdpg > button:nth-child(1)"}
  (Session info: chrome=93.0.4577.82)

0:00:10.112571
False
Message: no such element: Unable to locate element: {"method":"css selector","s

### Script para extraer los comentarios y sub-comentarios una vez desplegados todos los comentarios

In [40]:
time.sleep(5)
grand_father_id = 'B166OkVBPJR'
user_names = []
user_comments = []
comment_dates = []
likes_comment = []
child_comment_ids = []
father_comment_ids = []
post=[]
comment_overall = driver.find_elements_by_class_name('Mr508 ')
comment_id = 0
for c in comment_overall:
    comment = c.find_element_by_class_name('gElp9 ')
    container = comment.find_element_by_class_name('C4VMK')
    name = container.find_element_by_class_name('_6lAjh').text
    content = container.find_elements_by_tag_name('span')
    data_container = container.find_element_by_class_name('_7UhW9  ')
    date = data_container.find_elements_by_class_name('FH9sR')[0].get_attribute('datetime')
    likes = data_container.find_elements_by_class_name('FH9sR')[1].text
    try:
        replies = c.find_element_by_xpath("//span[contains(text(), 'View replies')]").click()

    except Exception as e:
        pass
    try:
        subcomments_container = c.find_element_by_tag_name('ul')
        subcomment = subcomments_container.find_elements_by_class_name('gElp9 ')
        subcomment_id = 0
        for i in subcomment:
            container = subcomments_container.find_element_by_class_name('C4VMK')
            name_subcomment = subcomments_container.find_element_by_class_name('_6lAjh').text
            content_subcomment = container.find_elements_by_tag_name('span')
            data_container_subcomment = container.find_element_by_class_name('_7UhW9  ')
            date_subcomment = data_container_subcomment.find_elements_by_class_name('FH9sR')[0].get_attribute('datetime')
            likes_subcomment = data_container_subcomment.find_elements_by_class_name('FH9sR')[1].text
            user_names.append(name_subcomment)
            user_comments.append(content_subcomment[1].text)
            comment_dates.append(date_subcomment)
            child_comment_ids.append(subcomment_id)
            father_comment_ids.append(comment_id)
            post.append(postURL)
            if likes_subcomment != 'Reply':
                likes_comment.append(likes_subcomment)
            else:
                likes_comment.append("0 likes")
            subcomment_id += 1
    except Exception as e:
        pass
    # content = content.replace('\n', ' ').strip().rstrip()
    post.append(postURL)
    user_names.append(name)
    user_comments.append(content[1].text)
    comment_dates.append(date)
    child_comment_ids.append(comment_id)
    father_comment_ids.append(grand_father_id)
    if likes != 'Reply':
        likes_comment.append(likes)
    else:
        likes_comment.append("0 likes")
    comment_id += 1

In [44]:
df_ig = pd.DataFrame({'Post': post, 'Caption': user_comments, 'Date': comment_dates, 'likesComment': likes_comment, 'IdFatherComment': father_comment_ids, 'idChildComment': child_comment_ids,'Username': user_names,})
print(df_ig)
# driver.close()

                                          Post  \
0     https://www.instagram.com/p/B166OkVBPJR/   
1     https://www.instagram.com/p/B166OkVBPJR/   
2     https://www.instagram.com/p/B166OkVBPJR/   
3     https://www.instagram.com/p/B166OkVBPJR/   
4     https://www.instagram.com/p/B166OkVBPJR/   
...                                        ...   
1151  https://www.instagram.com/p/B166OkVBPJR/   
1152  https://www.instagram.com/p/B166OkVBPJR/   
1153  https://www.instagram.com/p/B166OkVBPJR/   
1154  https://www.instagram.com/p/B166OkVBPJR/   
1155  https://www.instagram.com/p/B166OkVBPJR/   

                                                Caption  \
0                           @emilenevilasboas get ready   
1                                    @levinhonainternet   
2                                           @nuray.madi   
3     I would literally do that every time Lady and ...   
4                                            @joeblan25   
...                                          

In [53]:
df_ig.to_csv('scrapping.csv', index=False)

In [46]:
df_ig.to_excel("scrapping.xlsx")